In [5]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

In [9]:
data = pd.read_csv("data/sample_results_2.csv")
df = pd.DataFrame(data)
message = "\n".join(df[df.columns[1]].to_list())

In [ ]:
# Obtain SA Prediction
def get_SA_prediction(df, model, filename):
    texts = df[df.columns[1]].to_list()

    new_df = pd.DataFrame(columns=["Label", "Score"])

    for index in range(len(texts)):
        preds = model(texts[index])
        print(preds)
        pred_sentiment = preds[0]["label"]
        pred_score = preds[0]["score"]

        # write data into df
        new_df.at[index, "Label"] = pred_sentiment
        new_df.at[index, "Score"] = pred_score

    new_df.to_csv("data/{}".format(filename), index=False)
    results = new_df
    return results

model = "siebert/sentiment-roberta-large-english"
nlp = pipeline("sentiment-analysis", model=model, tokenizer=model)
results = get_prediction(df, nlp, "predict.csv")

In [ ]:
# Obtain text summarization model

def get_summary(text, model, filename):
    result = summarizer(text, max_length=130, min_length=30, do_sample=False)
    
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [57]:
def get_prediction(texts, model):

    new_df = pd.DataFrame(columns=["Content","Label", "Score"])

    for index in range(len(texts)):
        preds = model(texts[index])
        print(preds)
        pred_sentiment = preds[0]["label"]
        pred_score = preds[0]["score"]

        # write data into df
        new_df.at[index, "Label"] = pred_sentiment
        new_df.at[index, "Score"] = pred_score
        # write text
        new_df.at[index, "Content"] = "".join((texts[index]))

    new_df.to_csv("data/results.csv", index=False)
    results = new_df
    return results

def predict(filename):
    data = pd.read_csv('data/{}'.format(filename))
    texts = data.values.tolist()
    message = ""
    for i in texts:
        message += "".join(i)
    print(message)
    # results = get_prediction(texts, nlp)

predict("news.csv")

PARIS: Scientists said Saturday (Sep 10) they had identified the mechanism through which air pollution triggers lung cancer in non-smokers, a discovery one expert hailed as "an important step for science - and for society".
The research illustrated the health risk posed by the tiny particles produced by burning fossil fuels, sparking fresh calls for more urgent action to combat climate change.
It could also pave the way for a new field of cancer prevention, according to Charles Swanton of the UK's Francis Crick Institute.
Swanton presented the research, which has not yet been published in a peer-reviewed journal, at the European Society for Medical Oncology's annual conference in Paris.
Air pollution has long been thought to be linked to a higher risk of lung cancer in people who have never smoked.
"But we didn't really know whether pollution was directly causing lung cancer - or how," Swanton told AFP.
Traditionally it has been thought that exposure to carcinogens, such as those in ci